In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.cuda.device_count()
print("Number of GPUs available:", device)

Number of GPUs available: 1


In [ ]:
import torch


print("PyTorch CUDA Version:", torch.version.cuda)

print("Current CUDA Device:", torch.cuda.get_device_name(0))

!nvcc --version

PyTorch CUDA Version: 12.1
Current CUDA Device: NVIDIA L4
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.model_selection import ParameterGrid

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/DeepLearningProject/chest_xray/chest_xray'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def initialize_model(num_classes):
    model_ft = models.densenet121(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    return model_ft

criterion = nn.CrossEntropyLoss()

param_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.1],
    'momentum': [0.8, 0.9, 0.99],
    'batch_size': [16, 32, 64],
    'step_size': [5, 7, 10],
    'gamma': [0.1, 0.5]
}

def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

def test_model(model, dataloaders):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / dataset_sizes['test']
    return test_acc

best_acc = 0.0
best_params = None
best_model = None

for params in ParameterGrid(param_grid):
    print(f"Training with params: {params}")

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=params['batch_size'],
                                                  shuffle=True, num_workers=4)
                   for x in ['train', 'val', 'test']}

    model_ft = initialize_model(len(class_names)).to(device)

    optimizer_ft = optim.SGD(model_ft.parameters(), lr=params['lr'], momentum=params['momentum'])
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=params['step_size'], gamma=params['gamma'])

    model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, num_epochs=15)

    test_acc = test_model(model_ft, dataloaders)
    print(f"Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        best_params = params
        best_model = copy.deepcopy(model_ft)

print(f"Best params: {best_params}")
print(f"Best test accuracy: {best_acc:.4f}")

best_path = '/content/drive/MyDrive/DeepLearningProject/Saved_Models/DenseNet_model_best_Parameters.pth'
torch.save(best_model.state_dict(), best_path)


Training with params: {'batch_size': 16, 'gamma': 0.1, 'lr': 0.0001, 'momentum': 0.8, 'step_size': 5}
Epoch 0/14
----------
train Loss: 0.4541 Acc: 0.7917


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


val Loss: 0.6681 Acc: 0.5625

Epoch 1/14
----------
train Loss: 0.3199 Acc: 0.8652
val Loss: 0.6768 Acc: 0.5625

Epoch 2/14
----------
train Loss: 0.2600 Acc: 0.8933
val Loss: 0.7631 Acc: 0.6250

Epoch 3/14
----------
train Loss: 0.2333 Acc: 0.9039
val Loss: 0.6828 Acc: 0.6250

Epoch 4/14
----------
train Loss: 0.2153 Acc: 0.9149
val Loss: 0.6269 Acc: 0.6250

Epoch 5/14
----------
train Loss: 0.2117 Acc: 0.9185
val Loss: 0.7477 Acc: 0.6250

Epoch 6/14
----------
train Loss: 0.2107 Acc: 0.9121
val Loss: 0.7056 Acc: 0.6250

Epoch 7/14
----------
train Loss: 0.2138 Acc: 0.9141
val Loss: 0.7661 Acc: 0.6250

Epoch 8/14
----------
train Loss: 0.2114 Acc: 0.9162
val Loss: 0.6378 Acc: 0.6250

Epoch 9/14
----------
train Loss: 0.2051 Acc: 0.9162
val Loss: 0.7661 Acc: 0.6250

Epoch 10/14
----------
train Loss: 0.2071 Acc: 0.9177
val Loss: 0.6434 Acc: 0.6250

Epoch 11/14
----------
train Loss: 0.2075 Acc: 0.9176
val Loss: 0.7215 Acc: 0.6250

Epoch 12/14
----------
train Loss: 0.2043 Acc: 0.9166
v

KeyboardInterrupt: 